In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

# Atsisiunciame duomenis ( pvz . NVDA ir BTC )
data = yf.download(["BNB-USD", "DOGE-USD"], start="2018-01-01")["Close"]
returns = data.pct_change().dropna()


[*********************100%***********************]  2 of 2 completed


In [2]:
print(returns)
print()
print(returns.mean())

Ticker       BNB-USD  DOGE-USD
Date                          
2018-01-02  0.050289  0.026490
2018-01-03  0.078992  0.019136
2018-01-04 -0.033756  0.034764
2018-01-05  0.618973  0.261613
2018-01-06  0.525742  0.221583
...              ...       ...
2026-02-17 -0.014378 -0.003476
2026-02-18 -0.021112 -0.021047
2026-02-19  0.004361 -0.002264
2026-02-20  0.030476  0.018649
2026-02-21  0.003386  0.001395

[2973 rows x 2 columns]

Ticker
BNB-USD     0.002606
DOGE-USD    0.003389
dtype: float64


In [3]:
print(data.isnull().sum())

Ticker
BNB-USD     0
DOGE-USD    0
dtype: int64


In [ ]:
def analyze_asset(ticker, start_date):
    """
    Kainos grafiką su indikatoriumi. - Candlestick
    95% VaR reikšmę.
    Expected Shortfall
    Sharpe koeficientą.
    """